# Tate and Weil Pairings
Notebook done by Giacomo Borin for the course *Advanced Number Theory* in *Università di Trento*.  
The first part is necessary to define the curve and some useful functions. We will also implement the miller's algorithm, so is possible to use it to improve the efficency. 
Later we will use these functions to implement the pairings, leaving space for some testing.

### Definition of the curve
First we define a function to evaluate the embedding degree for a finite field of characteristic `q` and an integer `l`, that is the minimum $k$ such that $E[l] \subset E(\mathbb{F}_{q^k})$

In [1]:
def embedding_degree(q,r):
    r"""
    Return the embedding degree of `q` over `l`
    
        INPUT:

        - ``q`` -- a positive prime

        - ``l`` -- a positive integer
        
        OUTPUT:

        An integer.
        Return an error if `q` is not prime
    """
    if q.is_prime():
        for k in NN:
            if (q**k -1)%r == 0 and k != 0:
                return k
    else:
        raise ValueError('%s is not prime' % q)

In [2]:
embedding_degree?

Then we create an elliptic curve on the field $\mathbb{F}_{q^k}$ defined by the equation $y^2 = x^3 +ax+b$

In [3]:
q = 23
l = 6
k = embedding_degree(q,l)
K.<g> = GF(q**k,'g')
a = -1
b = 0
E = EllipticCurve(K,[a,b])
E

Elliptic Curve defined by y^2 = x^3 + 22*x over Finite Field in g of size 23^2

Sage is capable of esecuting different calculation on the curve, for example:

In [4]:
P = E.gen(0)
P, P+P, P + 2*P, E.cardinality(), E.is_singular()

((15*g + 12 : 18*g + 8 : 1),
 (5*g + 17 : 18*g + 8 : 1),
 (3*g + 17 : 5*g + 15 : 1),
 576,
 False)

*Observe that the points are always in affine form*

### Line and functions
Now we define two function to evaluate the line passing through some points, in particular `line_for` calculate the value on the line used in the definition of the group addition

In [5]:
def v_line_for(P,VAL):
    r"""
    Return the value of the vertical line passing through P on the point `VAL`
    
        INPUT:

        - ``P`` -- a point on the elliptic curve
        
        - ``VAL`` -- another point on the elliptic curve for the evaluation of the function

        OUTPUT:

        An element in the base field of the Elliptic curve
    """
    E = P.curve()
    # case for the trivial line
    if P == E(0):
        return 1
    else: 
        return VAL[0]-P[0]

In [6]:
def line_for(P,Q,VAL):
    r"""
    Return the value of the line through P and Q on the point `VAL`
    
        INPUT:

        - ``P,Q`` -- two points on the elliptic curve
        
        - ``VAL`` -- another point on the elliptic curve for the evaluation of the function

        OUTPUT:

        An element in the base field of the Elliptic curve
    """
    E = P.curve()
    if P != Q:
        # first two cases are the ones with a vertical one
        # observe that the situation with P and Q both zero is only possible in the last else
        if Q == E(0) or P+Q == E(0):
            return v_line_for(P,VAL)
        elif P == E(0):
            return v_line_for(Q,VAL)
        # normal case of a line for two distinct points
        else:
            # lam is the angular coefficent of the affine line
            lam = (Q[1] - P[1])/(Q[0] - P[0])
            return VAL[1] - P[1] - lam * (VAL[0] - P[0])
    else:
        # case of a vertical line
        # observe that this case absorb the situation of P and Q both zero
        if P[1]==0:
            return v_line_for(P,VAL)
        else:
            # here I recall the coefficent of the curve
            a1, a2, a3, a4, a6 = E.a_invariants()
            # lam is the angular coefficent of the affine line
            # observe that in this case the line is the tangent line to E in P
            lam = (3*P[0]**2 + 2*a2*P[0] + a4 - a1*P[1])/(2*P[1] + a1*P[0] + a3)
            return VAL[1]-P[1] - lam*(VAL[0]-P[0])

In [7]:
# some tests:
P = E.random_point()
Q = E.random_point()
VAL = E.random_point()
line_for(P,Q,VAL),line_for(P,P,Q)

(2*g + 3, 10*g + 16)

### Definiton of the rational function $f_{m,P}$
We now need to evaluate the rational function $f_{m,P}$ such that $(f_{m,P}) = m(P) - ([m]P) - \{m-1\}(O)$.  
*Observe that $O$ is the zero point (or infinity point).*  

First we will evaluate the function using the relation seen in the lectures:
$$ f_{(m,P)} = f_{(m-1,P)} \frac{l_{[m-1]P,P}}{v_{[m]P}} $$  
We assume that $f_{0,P}$ is the $1$ function.  
*Observe that if $P$ is in the torsion group $ E[m]$ than $(f_{m,P}) = m(P) - m(O)$*

In [8]:
def f(m,P,VAL):
    r"""
    Return the value of the function f_m,P on the point `VAL`
    
        INPUT:

        - ``m`` -- a positive integer

        - ``P`` -- a point on the elliptic curve that is not the zero point
        
        - ``VAL`` -- another point on the elliptic curve for the evaluation of the function

        OUTPUT:

        An element in the base field of the Elliptic curve
    """
    # errors
    if P == E(0):
        raise ValueError('P is the zero point')
    # base case (trivial function)
    if m == 1:
        return 1
    else:
        # induction
        f_m1 = f(m-1,P,VAL)
        # lines
        q1 = line_for((m-1)*P,P,VAL)
        q2 = v_line_for(m*P,VAL)
        return f_m1 * q1 / q2

This one is a naive implementation, but using the same principle of the *Russian moltiplication algorithm* is possible to improve the efficency. This implementation is known as the ***Miller's algotithm***.
In particular it uses the decomposition of the addition:
$$(f_{a+b,P}) = (f_{a,P}) + (f_{b,P}) + \{ ([a]P) + ([b]P) \} - \{([a+b]P) +(O)\} = (f_{a,P}) + (f_{b,P}) + (l_{[a]P,[b]P}) - (v_{[a+b]P})$$

In [9]:
def fm(m,P,VAL):
    r"""
    Return the value of the function f_m,P on the point `VAL` using a more efficent implementation
    
        INPUT:

        - ``m`` -- a positive integer

        - ``P`` -- a point on the elliptic curve that is not the zero point
        
        - ``VAL`` -- another point on the elliptic curve for the evaluation of the function

        OUTPUT:

        An element in the base field of the Elliptic curve
    """
    # errors
    if P == E(0):
        raise ValueError('P is the zero point')
    # a simple function that evaluate l(P,Q)/v(P+Q)
    def v_hm(P,Q,VAL):
        return line_for(P,Q,VAL)/v_line_for(P+Q,VAL)
    # T assignation
    T = P
    # base case
    fm = 1
    # write m in base 2
    mbits = m.bits()
    mbits.reverse()
    # intresting part
    for i in range(1,len(mbits)):
        # 'Doubling' of the function
        fm = (fm**2) * v_hm(T,T,VAL)
        # Doubling of the point
        T = 2*T
        if mbits[i]==1:
            # correction part necessary for the 1 bits
            fm = fm * v_hm(T,P,VAL)
            T = T + P
    return fm

In [10]:
# some test using the _miller_ function from sage
Q = E.random_element()
P = E.random_element()
P._miller_(Q,l) == f(l,P,Q),\
P._miller_(Q,l) == fm(l,P,Q)

(True, True)

In [11]:
fm?

## Weil Pairing
The Weil pairing can be defined in several ways, I've chose this one:  
Given two points $P,Q$ of $r$-torsion in the curve $E(\mathbb{F}_{q^k})$ consider the divisors $D_{P} = (P) - (O)$ and $D_{Q} = (Q) - (O)$, then the functions such that $(f_{r,P}) =  rD_P$ and $(f_{r,Q}) =  rD_Q$. 
Hence, if $\mu_r$ is the set of the $r$-root of unity, the Weil pairing $w_r:E[r] \times E[r] \to \mu_r$ is:  
$$ w_r(P,Q) = \frac{f_{r,P}(D_Q)}{f_{r,Q}(D_P)}$$

In [12]:
def b_weil_pairing(P1,Q1,r):
    r"""
        Compute the Weil pairing of P1 and Q1 using a random point to assist the evaluation

        INPUT:

        - ``P1,Q1`` -- two points on the same elliptic curve.

        - ``r`` -- a positive integer such that P1 and Q1 are of r-torsion

        OUTPUT:

        An `r`'th root of unity in the base field of the Elliptic curve
    """
    # check the errors
    if r*P1 != E(0):
        raise ValueError('The first point is not of %s-torsion' % r)
    if r*Q1 != E(0):
        raise ValueError('The second point is not of %s-torsion' % r)
    # case for (0:1:0)
    if P1 == E(0) or Q1 == E(0):
        return 1
    # interesting case
    while True:
        try:
            # This part is necessary to find a S necessary to evaluate the function on 
            # the divisors (Q)-(O) and (P)-(O)
            # In fact we want a point S such that S, Q1+S are not in the support of f_P1 and 
            # -S,Q1+S are not in the support of f_Q1 
            S = E.random_element()
            while S in [E(0) , P1 , -Q1 , P1-Q1]:
                S = E.random_element()
            # evaluation of the pairing
            num = f(r,P1,Q1+S)*f(r,Q1,-S)
            den = f(r,Q1,P1-S)*f(r,P1,S)
            # uncomment for a more efficent implementetion with miller's algorithm
            # num = fm(r,P1,Q1+S)*fm(r,Q1,-S)
            # den = fm(r,Q1,P1-S)*fm(r,P1,S)
            return num/den 
        except ZeroDivisionError:
            # this try catch is necessary in the case that during the evaluation of f we get a division by zero
            # for example if S is [j]P1 or -[j]P1 with 0 < j < l (very rare)
            print('Error',S)  

In [13]:
b_weil_pairing?

### Test area
The first test is a simple check for the base properties of the pairings

In [14]:
Q = E.random_element()
P = E.random_element()
R = E.random_element()
# very rude methood to get a point of given order, sometimes it does not work
P = (P.order()//l)*P
Q = (Q.order()//l)*Q
R = (R.order()//l)*R
a,b = randint(1,100), randint(1,100)
b_weil_pairing(P,Q+R,l),\
b_weil_pairing(P,Q,l)* b_weil_pairing(P,R,l),\
b_weil_pairing(P+Q,R,l),\
b_weil_pairing(P,R,l)* b_weil_pairing(Q,R,l),\
b_weil_pairing(a*P,b*R,l),\
b_weil_pairing(P,R,l)^(a*b)

(1, 1, 19*g + 16, 19*g + 16, 1, 1)

The result of this simple test are: 
* the Weil pairng
* if the result is a rooth of unity (as expected)
* if the result is the same of the native sage implementation

In [16]:
Q = E.random_element()
P = E.random_element()
P = (P.order()//l)*P
Q = (Q.order()//l)*Q
(b_weil_pairing(P,Q,l)),\
(P.weil_pairing(Q,l))**l==1,\
P.weil_pairing(Q,l) == b_weil_pairing(P,Q,l)

(22, True, True)

#### space for more test

In [17]:
Q = E.random_element()
P = E.random_element()
R = E.random_element()
P = (P.order()//l)*P
Q = (Q.order()//l)*Q
R = (R.order()//l)*R
a,b = randint(1,100), randint(1,100)

## Tate pairing
Given the point $P \in E[r]$ and $Q$ a representative of a coset in $E/rE$, where $E$ is the curve $E(\mathbb{F}_{q^k})$ ($k$ is the embedding degree) the divisor $D_{Q} = (Q) - (O)$ and the function such that $(f_{r,P}) =  r(P) - r(O)$.  
Then the Tate pairing $t_r:E[r] \times E/rE \to \mathbb{F}_{q^k}^* / (\mathbb{F}_{q^k}^*)^r $ is:  
$$ t_r(P,Q) = f_{r,P}(D_Q)$$
Since the result is a coset representative in order to obtain uniqueness we exponentiate the result by $(q^k -1)/r$, so we obtain what is sometimes called the *modified Tate pairng*

In [18]:
def b_tate_pairing(P1,Q1,r,k):
    r"""
        Compute the Tate pairing of P1 and Q1 using a random point to assist the evaluation

        INPUT:

        - ``P1,Q1`` -- two points on the same elliptic curve.

        - ``r`` -- a positive integer `r` such that P1 is of r-torsion
        
        - ``k`` -- the embedding degree of r (positive integer)

        OUTPUT:

        A unique element in the base field of the Elliptic curve
    """
    # errors
    if r*P1 != E(0):
        raise ValueError('The first point is not of %s-torsion' % r)
    if P1 == E(0) or Q1 == E(0):
        return 1
    while True:
        try:
            # This part is necessary to find a S necessary to evaluate the function on the divisor (Q)-(O)
            # In fact we want a point S such that S and Q1+S are not in the support of f_P1
            S = E.random_element()
            while (S in [E(0) , P1 , -Q1 , P1-Q1]):
                S = E.random_element()
            # here we evaluate f_P on (Q1+S) - (S) ~  (Q1) - (O)
            d = f(r,P1,Q1 + S) / f(r,P1,S)
            # uncomment for a more efficent implementetion with miller's algorithm
            # d = fm(r,P1,Q1 + S) / fm(r,P1,S)
            # we raise d to the power (q**k - 1)/r to get a unique result, but it is not necessary
            d = d ** ((q**k - 1)/r)
            return d
        except ZeroDivisionError:
            # this try catch is necessary in the case that during the evaluation of f we get a division by zero
            # for example if S is [j]P1 or -[j]P1 with 0 < j < l (very rare)
            print('Error',S)    

### Test area
The first test is a simple check for the base properties of the pairings. The second is to see that it does not depend on the representative. 

In [19]:
Q = E.random_element()
P = E.random_element()
R = E.random_element()
# very rude methood to get a point of given order, sometimes it does not work
P = (P.order()//l)*P
a,b = randint(1,100), randint(1,100)
b_tate_pairing(P,Q+R,l,k),\
b_tate_pairing(P,Q,l,k)* b_tate_pairing(P,R,l,k),\
b_tate_pairing(a*P,b*R,l,k),\
b_tate_pairing(P,R,l,k)^(a*b)

(19*g + 15, 19*g + 15, 1, 1)

In [20]:
Q = E.random_element()
P = E.random_element()
R = E.random_element()
P = (P.order()//l)*P
b_tate_pairing(P,Q,l,k),\
b_tate_pairing(P,Q+l*R,l,k)

(4*g + 8, 4*g + 8)

The result of this simple test are: 
* the Tate pairng
* if the result is the same of the native sage implementation

In [21]:
Q = E.random_element()
P = E.random_element()
P = (P.order()//l)*P
b_tate_pairing(P,Q,l,k),\
P.tate_pairing(Q,l,k) == b_tate_pairing(P,Q,l,k)

(22, True)

#### space for more test

In [22]:
Q = E.random_element()
P = E.random_element()
R = E.random_element()
P = (P.order()//l)*P
a,b = randint(1,100), randint(1,100)

### End of the implementation
*A little note.* Is possible that you have obtained some errors during the test: this is due to the rude methood used to get point of $l$-torsion. This is intentional because it is the fastest way to get these points and I'd like you to see that the functions are capable to raise errors for these cases.    
Hoping that you had fun using this script.  
Best regards,  
Giacomo Borin